# Groundedness Evaluator

답변이 주어진 문맥(context)에 기반하여 정확한지 평가하는 Evaluator 입니다.

이 Evaluator 는 RAG 의 답변에 대한 할루시네이션(Hallucination)을 평가하는데 활용할 수 있습니다.

이번 튜토리얼에서는 Upstage Groundness Checker와 직업 커스텀하여 만든 Groundness Checker를 활용하여 Groundedness를 평가하는 방법을 살펴보겠습니다.

In [ ]:
# 설치
# !pip install -qU langsmith langchain-teddynote

In [ ]:
# API KEY를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API KEY 정보로드
load_dotenv()

In [ ]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install -qU langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("CH16-Evaluations")

## RAG 성능 테스트를 위한 함수 정의

테스트에 활용할 RAG 시스템을 생성하겠습니다.

In [ ]:
from myrag import PDFRAG


# 질문에 대한 답변하는 함수를 생성
def ask_question_with_llm(llm):
    # PDFRAG 객체 생성
    rag = PDFRAG(
        "data/SPRI_AI_Brief_2023년12월호_F.pdf",
        llm,
    )

    # 검색기(retriever) 생성
    retriever = rag.create_retriever()

    # 체인(chain) 생성
    rag_chain = rag.create_chain(retriever)

    def _ask_question(inputs: dict):
        # 질문에 대한 컨텍스트 검색
        context = retriever.invoke(inputs["question"])
        # 검색된 문서들을 하나의 문자열로 결합
        context = "\n".join([doc.page_content for doc in context])
        # 질문, 컨텍스트, 답변을 포함한 딕셔너리 반환
        return {
            "question": inputs["question"],
            "context": context,
            "answer": rag_chain.invoke(inputs["question"]),
        }

    return _ask_question

In [ ]:
from langchain_openai import ChatOpenAI

gpt_chain = ask_question_with_llm(ChatOpenAI(model="gpt-4o-mini", temperature=0))

## UpstageGroundednessCheck

업스테이지(Upstage) 의 Groundedness Check 기능을 활용하기 위해서는 아래 링크에서 API 키를 발급받아야 합니다.

- [API 키 발급](https://console.upstage.ai/api-keys)

In [ ]:
from langchain_upstage import UpstageGroundednessCheck

# 업스테이지 Groundness Checker 생성
upstage_groundedness_check = UpstageGroundednessCheck()

In [ ]:
# Groundness Checker 를 실행하여 평가
request_input = {
    "context": "테디의 성별은 남자이며, 테디노트 유튜브 채널을 운영하고 있습니다.",
    "answer": "테디는 남자다.",
}

response = upstage_groundedness_check.invoke(request_input)
print(response)

In [ ]:
# Groundness Checker 를 실행하여 평가
request_input = {
    "context": "테디의 성별은 남자이며, 테디노트 유튜브 채널을 운영하고 있습니다.",
    "answer": "테디는 여자다.",
}

response = upstage_groundedness_check.invoke(request_input)
print(response)

UpstageGroundednessCheck Evaluator 를 정의합니다. 추후 Evaluate 함수에서 활용합니다.

In [ ]:
from langsmith.schemas import Run, Example
from langsmith.evaluation import evaluate


def upstage_groundness_check_evaluator(run: Run, example: Example) -> dict:
    # LLM 생성 답변, 정답 답변 가져오기
    answer = run.outputs.get("answer", "")
    context = run.outputs.get("context", "")

    # Groundness 체크
    groundedness_score = upstage_groundedness_check.invoke(
        {"answer": answer, "context": context}
    )
    groundedness_score = groundedness_score == "grounded"

    return {"key": "groundness_score", "score": int(groundedness_score)}

## langchain_teddynote Groundness Checker

OpenAI 의 모델을 활용하여 커스텀한 Groundness Checker 를 생성합니다.

OpenAI 모델을 활용하여 Groundedness 를 체크합니다.

In [ ]:
from langsmith.schemas import Run, Example
from langchain_teddynote.evaluator import GroundnessChecker
from langchain_openai import ChatOpenAI

# teddynote Groundness Checker 생성
groundedness_check = GroundnessChecker(
    ChatOpenAI(model="gpt-4o-mini", temperature=0)
).create()


def teddynote_groundness_check_evaluator(run: Run, example: Example) -> dict:
    # LLM 생성 답변, 정답 답변 가져오기
    answer = run.outputs.get("answer", "")
    context = run.outputs.get("context", "")

    # Groundness 체크
    groundedness_score = groundedness_check.invoke(
        {"answer": answer, "context": context}
    )
    groundedness_score = groundedness_score.score == "yes"

    return {"key": "groundness_score", "score": int(groundedness_score)}

Groundedness 평가를 실행합니다.

In [ ]:
from langsmith.evaluation import evaluate

# 데이터셋 이름 설정
dataset_name = "RAG_EVAL_DATASET"

# 실행
experiment_results = evaluate(
    gpt_chain,
    data=dataset_name,
    evaluators=[
        upstage_groundness_check_evaluator,
        teddynote_groundness_check_evaluator,
    ],
    experiment_prefix="GROUNDEDNESS-EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "Upstage & teddynote Groundness Checker 를 활용한 Hallucination 평가",
    },
)

![](./assets/output-09.png)

## Summary Evaluators 를 활용한 데이터셋에 대한 종합 평가

데이터셋 전체에 대한 Groundedness 평가를 실행할 때 유용합니다. (이전 단계는 개별 데이터에 대한 평가를 수행하였습니다.)

In [ ]:
from typing import List
from langsmith.schemas import Example, Run


def upstage_groundness_check_summary_evaluator(
    runs: List[Run], examples: List[Example]
) -> dict:
    def is_grounded(run: Run) -> bool:
        context = run.outputs["context"]
        answer = run.outputs["answer"]
        return (
            upstage_groundedness_check.invoke({"context": context, "answer": answer})
            == "grounded"
        )

    groundedness_scores = sum(1 for run in runs if is_grounded(run))
    return {"key": "groundness_score", "score": groundedness_scores / len(runs)}


def teddynote_groundness_check_summary_evaluator(
    runs: List[Run], examples: List[Example]
) -> dict:
    def is_grounded(run: Run) -> bool:
        context = run.outputs["context"]
        answer = run.outputs["answer"]
        return (
            groundedness_check.invoke({"context": context, "answer": answer}).score
            == "yes"
        )

    groundedness_scores = sum(1 for run in runs if is_grounded(run))
    return {"key": "groundness_score", "score": groundedness_scores / len(runs)}

In [ ]:
from langsmith.evaluation import evaluate

# 평가 실행
experiment_result1 = evaluate(
    gpt_chain,
    data=dataset_name,
    summary_evaluators=[
        upstage_groundness_check_summary_evaluator,
    ],
    experiment_prefix="GROUNDNESS_UPSTAGE_SUMMARY_EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "Upstage Groundness Checker 를 활용한 Hallucination 평가",
    },
)

# 평가 실행
experiment_result2 = evaluate(
    gpt_chain,
    data=dataset_name,
    summary_evaluators=[
        teddynote_groundness_check_summary_evaluator,
    ],
    experiment_prefix="GROUNDNESS_TEDDYNOTE_SUMMARY_EVAL",
    # 실험 메타데이터 지정
    metadata={
        "variant": "Teddynote Groundness Checker 를 활용한 Hallucination 평가",
    },
)

![](./assets/output-10.png)